<h1 style="text-align: center; font_size: 50px; color: green;">Song recommender </h1>

## Goal for today

look for differnt playlists/albums etc from varied genres
varied genres will improve the clustering machine learning we will do tomorrow

### Playlists
- everything good
- 80s
- hip hop


### Ideas and Improvements
#### General
- Formatting text
    - Check for ways to nicely format text  with bolds, headings etc
    - check if buttons can be added
    - use html to format and colour text. Eg. colou users chosen song as green
- Make function download latest hot 100 so it is always checking latest
- when not in hot 100 recommend song from alt 250 [done]
- ask would you like to enter another song? [done]
- user can choose number of recommendatiosn they would like
- Investigate how i could integrate work out data, heart rate, running speed etc
- Bonus feature: workout playlist creator: input genre, bpm, some sample songs and reccomender will recommend 10 songs for your workout playlist
- add plugins for jupyter lab to colapse code  boxes
- if user asks for a second recommendation ensure the same song is not recommendded [priority low]
- add artist and album info
- see if program can automatically compile playlist
  

#### Error handling
- If user enters an artists name, prompt them with "that is an artist not a song, we found this song by {artist} would you like to select this song?

- for fuzzy matching, ask user "did you mean..."
    -user selects y/n

## To do
#### Priority 


#### Bonus
- research what each of the audio features do

#### Tidying up
- fix "would you like to continue" after providing hot 100 recommendation
- fix "did you mean" to allow user response (y/n), if no, prompt user for new input
- ensure song reccomender does not reccomender the same song chosen by the user
- did you mean? - no - search for reccomendation in database
- did you mean? adjust threshold. currently its finding matches too easily


In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import random
from fuzzywuzzy import process
from IPython.display import display, HTML
import config
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
from IPython.display import IFrame
import pickle
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [3]:
def load(filename = "filename.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
        
    except FileNotFoundError: 
        print("File not found!")

def play_song(track_id):
    iframe = IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )
    display(iframe)
    
def find_track_id(song_name):
    results = sp.search(q=song_name,limit=1,market="GB")
    track_id = results["tracks"]["items"][0]["id"]
    return track_id


'''
def recommend_similar_song(original_song_track_id, scaler, kmeans_model):
    af_df = pd.DataFrame(sp.audio_features(original_song_track_id)).drop(columns=["mode", "key", "duration_ms", "liveness"])
    X = af_df.select_dtypes(include=['number'])
    X_scaled = scaler.transform(X)  # This already returns a NumPy array
    cluster_int = kmeans_model.predict(X_scaled)[0]  # Directly use the NumPy array for prediction
    recommended_song_af = audio_features_db_df[audio_features_db_df['cluster'] == cluster_int].sample()
    recommended_song_track_id = recommended_song_af["id"].item()
    return recommended_song_track_id


def recommend_similar_song(original_song_track_id, scaler, kmeans_model):
    af_df = pd.DataFrame(sp.audio_features(original_song_track_id)).drop(columns=["mode", "key", "duration_ms", "liveness"])
    X = af_df.select_dtypes(include=['number'])
    X_scaled = scaler.transform(X) 
    pd.DataFrame(X_scaled)
    X_scaled_df = pd.DataFrame(X_scaled, columns = X.columns)
    clusters_array = kmeans_model.predict(X_scaled_df)
    cluster_int = clusters_array[0]
    recommended_song_af = audio_features_db_df[audio_features_db_df['cluster'] == cluster_int].sample()
    recommended_song_track_id = recommended_song_af["id"].item()
    return recommended_song_track_id

'''
def recommend_similar_song(original_song_track_id, scaler, pca, kmeans_model, audio_features_db_df):
    # Fetch audio features for the original song
    af_df = pd.DataFrame(sp.audio_features(original_song_track_id))
    
    # Select numerical features and scale them
    X = af_df.select_dtypes(include=['number']).drop(columns=["mode", "key", "duration_ms", "liveness"])
    X_scaled = scaler.transform(X)
    
    # Transform features using already fitted PCA
    X_pca = pca.transform(X_scaled)
    
    # Predict the cluster for the transformed features
    cluster_int = kmeans_model.predict(X_pca)[0]
    
    # Sample a song from the same cluster, excluding the original song
    recommended_song_af = audio_features_db_df[(audio_features_db_df['cluster'] == cluster_int) & 
                                               (audio_features_db_df['id'] != original_song_track_id)].sample()
    
    # Return the track ID of the recommended song
    recommended_song_track_id = recommended_song_af["id"].item()
    return recommended_song_track_id

def recommend_hot_song(user_input, hot_100_df):
    # Convert all hot 100 song titles to lowercase for case-insensitive comparison
    songs_lower = hot_100_df['song'].str.lower()
    # Filter out the user's song and select a random recommendation from hot 100
    hot_recommendations = hot_100_df[~songs_lower.str.contains(user_input.lower())]
    if not hot_recommendations.empty:
        recommended_hot_song = random.choice(hot_recommendations['song'].values)
        print(f"\nHere's another song from the Hot 100: '{recommended_hot_song}'\nGive it a listen!")
        rec_hot_song_track_id = find_track_id(recommended_hot_song)
    return rec_hot_song_track_id
    

def would_you_like_to_select_another_song():
    while True:  # Loop until a valid response is received
        cont = input("\nWould you like to select another song? (yes/no):\n  Response: ").strip().lower()
        if cont in ["no", "n"]:
            print("\nThank you for using Gnod's Song Recommender. Goodbye!")
            return False  # User does not want to continue
        elif cont in ["yes", "y"]:
            return True  # User wants to continue
        else:
            print("Invalid response. Please answer with 'yes' or 'no'.")  # Ask again for a valid response

def song_recommender(hot_100_df, audio_features_db_df, scaler, kmeans_model):
    print("\n                                     Welcome to Gnod's Song Recommender!\nThe Recommender allows you to search for a song you like. It will then inform you if the song is in the Billboard Hot 100 or not and also provide you with a recommendation for another song\nNote: The song search is case insensitive.\n")
    # Convert all hot 100 song titles to lowercase for case-insensitive comparison
    songs_lower = hot_100_df['song'].str.lower()
    
    while True:
        # Get user input
        user_input = input("Please enter the name of a song.\n  Song: ").strip().title()
        # Find_track_id
        track_id = find_track_id(user_input)
        play_song(track_id)
        
        if not user_input:
            print("\nYou didn't enter a song. Please try again.\n  Song: ")
            continue
        
        try:       
            if user_input.lower() in songs_lower.values:
                print(f"\n'{user_input.title()}' is currently in the Billboard Hot 100.")                
                rec_hot_song_track_id = recommend_hot_song(user_input, hot_100_df)
                play_song(rec_hot_song_track_id)
                would_you_like_to_select_another_song()
                break  # Exit loop after providing a recommendation
            
            else:
                
                # Use fuzzy matching to find the closest match to user_input in the 'song' column
                result = process.extractOne(user_input, songs_lower, score_cutoff= 87)
                # score_cutoff is the minimum score threshold for considering a match
        
                if result: # A match was found with a score above the cutoff
                    closest_match, match_score, i = result
                    input_dym = input(f"\nDid you mean '{closest_match.title()}'? (yes/no)\n").strip().lower()
                    if input_dym == "yes":
                        print(f"\n'{closest_match.title()}' is currently in the Billboard Hot 100.")                
                        rec_hot_song_track_id = recommend_hot_song(closest_match, hot_100_df)
                        play_song(rec_hot_song_track_id)
                        would_you_like_to_select_another_song()
                        break
                else:
                    print(f"\n'Here's a song I think you'll like.\n")
                    original_song_track_id = find_track_id(user_input)
                    recommended_song_track_id = recommend_similar_song(original_song_track_id, scaler, kmeans_model)
                    play_song(recommended_song_track_id)
                    
        except KeyError:
            print("\nError: The DataFrame does not have a 'song' column.")
            break
        
        cont = input("\nWould you like to select another song?\n  Response: ").lower()
        if cont == "no":
            print("\nThank you for using Gnod's Song Recommender\nGoodbye")
            break



In [4]:
#load datasets
hot_100_df = pd.read_csv('../data/hot_100.csv',index_col = 0)
alt_250 = pd.read_csv('../data/alt_250.csv',index_col = 0)
audio_features_db_df = pd.read_csv("../data/audio_features_db_df.csv",index_col = 0)

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))
#Load scalers and models
scaler= load(filename="model/scaler1.pickle")
kmeans_model = load(filename="model/kmeans_1.pickle")
pca = load(filename="model/pca.pickle")

In [ ]:
song_recommender(hot_100_df, audio_features_db_df, scaler, kmeans_model)


                                     Welcome to Gnod's Song Recommender!
The Recommender allows you to search for a song you like. It will then inform you if the song is in the Billboard Hot 100 or not and also provide you with a recommendation for another song
Note: The song search is case insensitive.



Please enter the name of a song.
  Song:  time to say goodbye



'Here's a song I think you'll like.




Would you like to select another song?
  Response:  yes
Please enter the name of a song.
  Song:  sunshine



'Here's a song I think you'll like.



...


'Here's a song I think you'll like.




Would you like to select another song?
  Response:  yes
Please enter the name of a song.
  Song:  animals - martin garrix



'Here's a song I think you'll like.




Would you like to select another song?
  Response:  yes
Please enter the name of a song.
  Song:  macarena



'Here's a song I think you'll like.



In [ ]:
recommended_song_af = audio_features_db_df[audio_features_db_df['cluster'] == 2].sample()
recommended_song_track_id = recommended_song_af["id"]
play_song(recommended_song_track_id)

In [ ]:
audio_features_db_df

In [ ]:
recommended_song_af = audio_features_db_df[audio_features_db_df['cluster'] == 1].sample()


In [ ]:
recommended_song_track_id = recommended_song_af["id"]

In [ ]:
recommended_song_track_id = find_track_id(recommended_song_af)
recommended_song_track_id

In [ ]:
play_song(recommended_song_track_id)

In [ ]:
 # Select numerical features and scale them
    X = af_df.select_dtypes(include=['number']).drop(columns=["mode", "key", "duration_ms", "liveness"])
    X_scaled = scaler.transform(X)
    
    # Transform features using already fitted PCA
    X_pca = pca.transform(X_scaled)

print(X_pca.shape)  # This should output something like (1, 3) for a single song

In [5]:
audio_features_db_df

,id,danceability,energy,loudness,speechiness,acousticness,instrumentalness,valence,tempo,time_signature,cluster
0,6yzJiHzsscf0n88xb9aNT4,0.382,0.451,-6.606,0.0293,0.5570,0.000034,0.506,150.803,4,24
1,5zdXRoKLZ54VBdbdn7yhz4,0.742,0.375,-13.829,0.0482,0.3970,0.901000,0.105,102.023,4,17
2,5qvCKAyjgHnpR5c63OedCY,0.585,0.552,-10.613,0.0353,0.1940,0.017100,0.496,100.089,4,12
3,2vByA9JgfMU2bPjUqmxCdv,0.405,0.535,-7.504,0.0531,0.0531,0.106000,0.637,97.698,4,10
4,0pPQwIjwqduvLfZORYYEaI,0.586,0.526,-6.928,0.0564,0.5550,0.000000,0.456,107.153,4,1
...,...,...,...,...,...,...,...,...,...,...,...
19463,6RWCvgNCEqXuVZYnDTPDEh,0.431,0.628,-6.259,0.0384,0.4030,0.000008,0.277,118.016,4,24
19464,4oEf84vBYVftf6KmZexhVo,0.486,0.640,-8.027,0.0551,0.1360,0.000076,0.289,122.017,4,24
19465,0lbHiQaHPlp5gGtsUPtj6l,0.606,0.757,-6.206,0.0302,0.0157,0.000205,0.588,103.051,4,28
19466,4kjI1gwQZRKNDkw1nI475M,0.455,0.621,-7.694,0.0517,0.6510,0.000003,0.280,119.043,4,1
